# Clase 10

Objetivos

1. Introducción a Cloud Computing.
2. Uso de jupyter en el supercomputador.
2. Alternativa a FBA que produce soluciones únicas: minFlux
3. Modelo que incorpora información transcriptómica: SPOT

# 1. Introducción a Cloud Computing


Revisemos esta pequeña [presentación](https://docs.google.com/presentation/d/12me-vn1Dsa5LktzO-Ivk4PKsLO4TXjBolbHP5OQT2HY/edit?usp=sharing) a modo de introducción.


# 2. Uso de jupyter en el supercomputador

La clase de hoy requiere de la librería cplex. Cplex es un software comercial para hacer optimización lineal, cuadrática, y entera. Cplex proporciona una licensia gratis para uso académico. La clase de hoy la realizaremos en el supercomputador de Amazon Web Services. En ese computadore cplex ya está instalado.

descarga el script [aws_jupyter_windows.sh](https://drive.google.com/file/d/1oG2UnlPMpACywDEbJG86KYixqSCEwFp3/view?usp=sharing) en tu computador y en el terminal ejecuta:

``` 
bash utem_jupyter_windows.sh -u username
```

Si usas Linux o Mac usa este script en vez: [aws_jupyter_linux.sh](https://drive.google.com/file/d/1_H8TLpI6XV98UUgoAxTKK7bi5Gyjonhb/view?usp=sharing). En tu computador y en el terminal ejecuta:

``` 
bash utem_jupyter_linux.sh -u username
```

In [ ]:
#Prueba que las siguientes librerias estén disponibles

import cobra.test
import cplex # usaremos el cluster UTEM
import pandas as pd
import numpy as np

# 3. minFLUX

La ventaja de minFlux sobre FBA es que se obtiene una solución única. MinFlux también elimina ciclos termodinámicamente infactibles. Sin embargo, minFlux también elimina ciclos que se han observado como activos. Por ejemplo, en *Escherichia coli*  elimina el ciclo del glyoxilato, aún cuando este se ha observado como [activo](https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0243067). Veamos una implementación de minFlux en Python.

In [ ]:
import optlang 
optlang.available_solvers

In [ ]:
#1. Compute biomass flux using FBA
model = cobra.test.create_test_model("ecoli")
solution_fba = model.optimize()

sumFluxes = 0
for reaction in model.reactions:
    sumFluxes += solution_fba[reaction.id]**2
print(model.objective.value, sumFluxes)

In [ ]:
#2. Run minFlux

In [ ]:
#2.1 Set v_biomass to the value found using FBA

model.reactions.BIOMASS_Ec_iJO1366_core_53p95M.bounds=(solution_fba["BIOMASS_Ec_iJO1366_core_53p95M"]
                                          ,solution_fba["BIOMASS_Ec_iJO1366_core_53p95M"])

In [ ]:
#2.1 Add Euclidean objecive function

for i, reaction in enumerate(model.reactions):
    if i == 0:
        euclidean_expression  = model.reactions.get_by_id(reaction.id).flux_expression**2
    else:
        euclidean_expression += model.reactions.get_by_id(reaction.id).flux_expression**2
        
euclidean_objective = model.problem.Objective(euclidean_expression,direction="min")
model.objective = euclidean_objective        

In [ ]:
#2.3 Compute new distribution of fluxes
solution_minFlux = model.optimize()

In [ ]:
sumFluxes = 0
for reaction in model.reactions:
    sumFluxes += solution_minFlux[reaction.id]**2
print(solution_minFlux["BIOMASS_Ec_iJO1366_core_53p95M"], model.objective.value, sumFluxes)

## Ejercicio 1

Predecir los flujos de *Saccharomyces ceverisiae* (iMM904) usando minFlux

# 4. SPOT

SPOT utiliza información transcriptómica para producir soluciones únicas. A diferencia de FBA y minFlux, SPOT no necesita un reacción de biomasa, lo cual es especialmente útil para celulas pluricelulares en donde maximizar biomasa no es típicamente una función objetivo apropiada.

Usemos como ejemplo los datos recopilados por et al [Min Kyung Kim et al](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0157101#sec021).

In [ ]:
# Load fpkm data and model

#wget -O S2.xls 'https://journals.plos.org/plosone/article/file?type=supplementary&id=info:doi/10.1371/journal.pone.0157101.s002'

data=pd.read_excel("S2.xls",sheet_name="Transcriptomic data - Holm", index_col=0, header=0) 
model = cobra.test.create_test_model("ecoli")

In [ ]:
# Define una función para cargar los fpkm de acuerdo a las gene_reaction_rules

def getG(grl,fpkm):
    grl=grl.replace("(","")
    grl=grl.replace(")","")
    g = 0
    for subrule in grl.split("or"):
        g_subrule = []
        for gene in subrule.split("and"):
            gene = gene.replace(" ","")
            if gene in fpkm:
                g_subrule.append( float(fpkm[gene]) )
        if len(g_subrule)>=1: 
            g += min(g_subrule)
    return(g)

In [ ]:
# Create a dictionary with the fpkm of each gene

fpkm={}

for gene in model.genes:
    if gene.id  in data.index:
        fpkm[gene] = data["Ref"].loc[gene.id]
    #else:
    #    print(gene.id," not found")

In [ ]:
# Create a dictionary with the fpkm for each reaction
g = {}
mean_g = np.mean(list(fpkm.values()) ) # useful when not a grl available

for reaction in model.reactions:
    if reaction.gene_reaction_rule == "":
        g[reaction.id] = mean_g    
    else:
        g[reaction.id] = getG(reaction.gene_reaction_rule,fpkm)

In [ ]:
#1 Define objecive function
model = cobra.test.create_test_model("ecoli")

for i, reaction in enumerate(model.reactions):
    if i == 0:
        dotProduct_expression  = model.reactions.get_by_id(reaction.id).forward_variable*g[reaction.id]
        dotProduct_expression += model.reactions.get_by_id(reaction.id).reverse_variable*g[reaction.id]
    else:
        dotProduct_expression += model.reactions.get_by_id(reaction.id).forward_variable*g[reaction.id]
        dotProduct_expression += model.reactions.get_by_id(reaction.id).reverse_variable*g[reaction.id]

In [ ]:
dotProduct_objective = model.problem.Objective(dotProduct_expression,direction="min")
model.objective = dotProduct_objective 

In [ ]:
#2.3 Compute new distribution of fluxes
solution_SPOT = model.optimize()

In [ ]:
solution_SPOT

# Tarea

Una alternativa a minFlux es [MaxEnt](https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0243067). Al igual que minFlux, MaxEnt también produce una solución única pero no necesita de la definición de una función biológica a optimizar (¿será esto útil para modelar el YMC?). También, MaxEnt es la base de Pheflux (el cual, como SPOT, incorpora información transcriptómica). Para el próximmo jueves proponer una implementación de [MaxEnt](https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0243067) en la red metabólica de *S. cerevisiae*.